# Hong Kong Immigration Data
Source: Immigration Dept Hong Kong 

by Kenton Kwok

https://github.com/kwokkenton/hongkong_immigration

In [1]:
# import relevant packages
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datapane as dp

KEY = ''
dp.login(KEY)


headers = ['Date', 'Control Point', 'Hong Kong Residents_Arrival','Mainland Visitors_Arrival','Other Visitors_Arrival', 
               'Hong Kong Residents_Departure', 'Mainland Visitors_Departure','Other Visitors_Departure']

def append_row(row, table, date):
    """ extracts data from one row of the web table and adds it to the big table
    """
    row.find_all(attrs={'headers': 'Hong_Kong_Residents_Arrival', 'headers': 'Hong_Kong_Residents_Departures'})

    # since format of the website is the same, can index locations corresponding to headers
    slices = [5,6,7,10,11,12]

    row_data = row.find_all('td')

    #extract control point

    control_pt = row_data[3].get_text()

    #extract number data
    entry = [date, control_pt]
    for s in slices: 
        num = row_data[s].get_text()
        num = int(num.replace(',',''))
        entry.append(num)

    # appends row entry to big table
    table.append(entry)
    

    return table
    

def get_soup(date):
    """ gets soup object from datetime object provided
    """
    
    #string format the date_string
    date_string = date.strftime('%Y%m%d')
    
    # request the static site
    URL = "https://www.immd.gov.hk/eng/stat_{}.html".format(date_string)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    return soup


def get_date_list(start, end):
    # generates all dates between start date and end date
    # in required format
    
    date_list = []
    delta = end - start      
    for i in range(delta.days + 1):
        day = start + timedelta(days=i)
        date_list.append(day)
    
    return date_list

def gen_table(start, end):
    """ iterates through all days and creates a table
    """
    # static government website in table format, with dates in URL
    base_URL = "https://www.immd.gov.hk/eng/stat_%s.html"
    table = []

    date_list = get_date_list(start, end)
    
    for day in date_list: 
        soup = get_soup(day)

        # exclude header and footer rows in soup, add new rows to table
        # each row should represent a separate control point
        try: 
            for row in soup.find_all('tr')[5:-2]:
                table = append_row(row, table, day)
        except:
            # catch all exception if format is wrong
            print('The date', day, 'could not be scraped.')

            
    return table

#for row in soup.find_all('tr')[5:-2]:
#    table = append_row(row, table)
#table

/Users/kenton/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Run this first time, as it initialises the Excel file.

In [2]:
#df_all = pd.DataFrame(gen_table(date(2020,1,24), date(2020,7,1)), columns=headers)
#df_all.to_csv('hkimm.csv', index=False)

---

### This section of the code scrapes future data and adds it to existing data.

In [3]:
df_old = pd.read_csv('hkimm.csv', parse_dates=['Date'])
# latest timestamp
start = max(df_old.Date.dt.to_pydatetime()).date()+timedelta(days=1)
end = date.today()-timedelta(days=1)
start, end

(datetime.date(2021, 7, 20), datetime.date(2021, 7, 19))

In [4]:
# generate new dataframe
if start > end: 
    pass
else:
    df_new = pd.DataFrame(gen_table(start, end), columns=headers) 
    df_old = df_old.append(df_new, ignore_index=True)
    df_old.to_csv('hkimm.csv', index=False)

# Data Cleaning

In [5]:
# read csv and parse datetimes
df = pd.read_csv('hkimm.csv', parse_dates=['Date'])
#df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')
df.columns = headers
df.head()


#reformat data in a better way
arrives = df[headers[0:5]].rename(columns={'Hong Kong Residents_Arrival': 'Hong Kong Residents', 
                                     'Mainland Visitors_Arrival': 'Mainland Visitors',
                                     'Other Visitors_Arrival': 'Other Visitors'
                                    })

departs = df[[*headers[0:2], *headers[5:]]].rename(columns={'Hong Kong Residents_Departure': 'Hong Kong Residents', 
                                     'Mainland Visitors_Departure': 'Mainland Visitors',
                                     'Other Visitors_Departure': 'Other Visitors'
                                    })
arrives = pd.melt(arrives, id_vars=['Date','Control Point'], value_name='Arrivals', var_name = 'Identity' )
departs = pd.melt(departs, id_vars=['Date','Control Point'], value_name='Departures', var_name = 'Identity' )['Departures']

df = arrives.join(departs)
df['Flux'] = df.Arrivals - df.Departures

In [6]:
df

,Date,Control Point,Identity,Arrivals,Departures,Flux
0,2020-01-24,Airport,Hong Kong Residents,20724,74876,-54152
1,2020-01-24,Express Rail Link West Kowloon,Hong Kong Residents,1717,20096,-18379
2,2020-01-24,Hung Hom,Hong Kong Residents,433,3498,-3065
3,2020-01-24,Lo Wu,Hong Kong Residents,26340,72074,-45734
4,2020-01-24,Lok Ma Chau Spur Line,Hong Kong Residents,16750,32215,-15465
...,...,...,...,...,...,...
23743,2021-07-19,Shenzhen Bay,Other Visitors,4,3,1
23744,2021-07-19,China Ferry Terminal,Other Visitors,0,0,0
23745,2021-07-19,Harbour Control,Other Visitors,0,0,0
23746,2021-07-19,Kai Tak Cruise Terminal,Other Visitors,0,0,0


---
# Exploratory Data Analysis

First we can look at the average number of arrivals and departures each day.

In [42]:
df.groupby(['Date']).sum().sum()
#df.groupby(['Control Point']).mean().sort_values('Arrivals', ascending = False)[['Arrivals', 'Departures']]

Arrivals      4014878
Departures    3610011
Flux           404867
dtype: int64

In [8]:
ad = df.groupby(['Control Point']).sum().sort_values('Arrivals', ascending = False)[['Arrivals', 'Departures']]
(ad.Arrivals + ad.Departures)/ ((ad.Arrivals + ad.Departures)).sum()


Control Point
Airport                           0.429344
Shenzhen Bay                      0.235561
Hong Kong-Zhuhai-Macao Bridge     0.110650
Lo Wu                             0.087043
Lok Ma Chau Spur Line             0.054405
Lok Ma Chau                       0.031292
Express Rail Link West Kowloon    0.016037
Macau Ferry Terminal              0.017113
Kai Tak Cruise Terminal           0.006440
China Ferry Terminal              0.004285
Sha Tau Kok                       0.002928
Hung Hom                          0.002554
Man Kam To                        0.002110
Harbour Control                   0.000239
Heung Yuen Wai                    0.000000
dtype: float64

In [38]:
# we can use the graph 
fig = px.bar(df.groupby(['Control Point']).sum().sort_values('Arrivals', ascending = False)[['Arrivals', 'Departures']], 
       barmode='group', labels={'variable':'Type', 'value': 'Number'},  title='Most frequently used Control Points')
fig.show()
fig.write_html("./graphs/0_control.html")
dp.Report(dp.Plot(fig)).upload(name = 'Control Point')

Publishing document and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/kwokkenton/reports/control-point/) to view your report and optionally share it with the Datapane Community

In [10]:
adi = df.groupby(['Identity']).sum()[['Arrivals', 'Departures']]
(adi.Arrivals + adi.Departures) / (adi.Arrivals + adi.Departures).sum()

Identity
Hong Kong Residents    0.809702
Mainland Visitors      0.111277
Other Visitors         0.079022
dtype: float64

In [37]:
fig = px.bar(df.groupby(['Identity']).sum()[['Arrivals', 'Departures']], barmode='group', 
       title='Arrivals and Departures by Identity')
fig.show()
fig.write_html("./graphs/0_iden.html")
dp.Report(dp.Plot(fig)).upload(name = 'Identity')

Publishing document and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/kwokkenton/reports/identity/) to view your report and optionally share it with the Datapane Community

In [36]:

both = df.groupby(['Identity', 'Control Point']).sum()[['Arrivals', 'Departures']]
both = (both.Arrivals+both.Departures).reset_index()
both.columns = ["Identity", 'Control Point', 'Traffic']
#px.parallel_categories(both, counts=0)
#both.sort_values(by='Identity', ascending=True)

fig = go.Figure(go.Parcats(
    dimensions=[
        {'label': 'Identity',
         'values': both.Identity.values},
        {'label': 'Control Point',
         'values': both['Control Point'].values}],
    counts=both.Traffic
))
fig.show()

fig.write_html("./graphs/0_sankey.html")
dp.Report(dp.Plot(fig)).upload(name = 'Sankey')

Publishing document and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/kwokkenton/reports/sankey/) to view your report and optionally share it with the Datapane Community

---
# Time Series Analysis

In [35]:
tot = df.sort_values('Date').groupby(['Date']).sum()[["Arrivals", "Departures"]]
fig = go.Figure()
fig.add_trace(go.Bar(x=tot.index, y=tot.Arrivals, name="Arrivals"))
fig.add_trace(go.Bar(x=tot.index, y=-tot.Departures, name="Departures"))
fig.update_layout(barmode='relative', title_text='Arrivals and Departures into Hong Kong')

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(count=6,
                     label="6m",
                     step="month",
                     stepmode="backward"),
                dict(count=1,
                     label="YTD",
                     step="year",
                     stepmode="todate"),
                dict(count=1,
                     label="1y",
                     step="year",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    ),
    yaxis=dict(
       autorange = True,
       fixedrange= False
   )
)




fig.add_vrect(
    x0=date(2020,1,24), x1=date(2020,2,7),
    fillcolor='LightPink', opacity=0.5,
    annotation_text="Chinese New Year Period", annotation_position="top left",
    layer="below", line_width=0,
)



fig.show()
fig.write_html("./graphs/1_time.html")
dp.Report(dp.Plot(fig)).upload(name = 'All time')

Publishing document and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/kwokkenton/reports/all-time/) to view your report and optionally share it with the Datapane Community

In [34]:
fig = px.line(df[df.Date > pd.Timestamp(date(2020,4,1))].sort_values('Date').groupby(['Date', 'Identity']).sum().reset_index(), 
              x="Date", y="Flux", color='Identity', 
              title="Flux of people in Hong Kong, by Identity")
fig.show()
fig.write_html("./graphs/1_iden.html")
dp.Report(dp.Plot(fig)).upload(name = 'HK Flux')

Publishing document and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/kwokkenton/reports/hk-flux/) to view your report and optionally share it with the Datapane Community

In [40]:
fig_arrive = px.line(df[df.Date > pd.Timestamp(date(2020,4,1))].sort_values('Date')[df.Identity == 'Hong Kong Residents']
        .groupby(['Date', 'Control Point']).sum().reset_index(), 
              x="Date", y="Arrivals", color='Control Point', 
              title="Arrivals of HK people in Hong Kong")
fig_arrive.show()
#fig.write_html("./graphs/1_arr.html")


/Users/kenton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [41]:
fig_depart = px.line(df[df.Date > pd.Timestamp(date(2020,4,1))].sort_values('Date')[df.Identity == 'Hong Kong Residents']
        .groupby(['Date', 'Control Point']).sum().reset_index(), 
              x="Date", y="Departures", color='Control Point', 
              title="Departures of HK people from Hong Kong")
fig_depart.show()
#fig.write_html("./graphs/1_dept.html")
dp.Report(dp.Plot(fig_arrive),
          dp.Plot(fig_depart)).upload(name = 'HK Arrivals and Departures')

/Users/kenton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Publishing document and associated data - *please wait...*

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/kwokkenton/reports/hk-arrivals-and-departures/) to view your report and optionally share it with the Datapane Community

# Cumulative Fluxes

In [31]:
iden = df[df['Date'] >= pd.Timestamp(date(2020,1,1))].groupby(['Date', 'Identity']).sum()
# advanced statement, transform select, apply
iden['Cumulative Flux'] = iden.groupby('Identity')['Flux'].transform(pd.Series.cumsum)



iden.reset_index(inplace=True) 
iden

fig = px.line(iden, x='Date', y="Cumulative Flux", color='Identity')


fig.add_vline(x=pd.Timestamp(date(2020,7,1)), line_width=2, line_dash="dash", line_color="gray")
fig.show()
fig.write_html("./graphs/2_iden.html")
dp.Report(dp.Plot(fig)).upload(name = 'Cumulative Flux 1')

Publishing document and associated data - *please wait...*

Your report only contains a single element - did you know you can include additional plots, tables and text in a report? Check out [the docs](https://docs.datapane.com/reports/blocks/layout-pages-and-selects) for more info

Your report doesn't contain any text - consider using TextReport to upload assets and add text to your report from your browser

Report successfully uploaded, click [here](https://datapane.com/u/kwokkenton/reports/cumulative-flux-1/) to view your report and optionally share it with the Datapane Community

In [21]:
#filter
iden = df[df['Date'] >= pd.Timestamp(date(2020,7,1))][df['Identity'] == 'Hong Kong Residents']
# advanced statement, transform select, apply
iden['Cumulative Flux'] = iden.groupby('Control Point')['Flux'].transform(pd.Series.cumsum)



iden.reset_index(inplace=True) 

fig = px.line(iden, x='Date', y="Cumulative Flux", color='Control Point')
fig.show()
fig.write_html("./graphs/2_control.html")

/Users/kenton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



# Other Improvements

## grouping the control points into nature
x = df.copy()
x = x.replace(['Lok Ma Chau Spur Line', 'Express Rail Link West Kowloon', 'Hung Hom'],
         'Rail')
x = x.replace(['Lo Wu', 'Lok Ma Chau','Sha Tau Kok', 'Shenzhen Bay', 'Man Kam To', 'Hong Kong-Zhuhai-Macao Bridge'],
         'Land')
x = x.replace(['China Ferry Terminal', 'Macau Ferry Terminal'],
         'Ferry')
x = x.replace(['Heung Yuen Wai', 'Harbour Control','Kai Tak Cruise Terminal'],
         'Other')
         
fig = px.line(x.sort_values('Date').groupby(['Date', 'Control Point']).sum().reset_index(), 
              x="Date", y="Arrivals", color='Control Point', 
              title="Flux (arrivals- departures) of people in Hong Kong by Control Point")
fig.show()
fig.write_html("flux.html")

df.sort_values('Date').groupby(['Date', 'Identity']).sum().reset_index()

hongkong = x[x.Identity == 'Hong Kong Residents']


mainland = x[x.Identity == 'Mainland Visitors']
mainland['Cumulative flux'] = mainland['Flux'].cumsum()

bars = df.copy()

bars['Departures'] *= -1
bars = bars.set_index('Date')
bars = bars.groupby([pd.Grouper(freq='m'), 'Identity']).sum()
px.bar(bars.groupby(['Date', 'Identity']).sum().reset_index(),
        x="Date", y=["Arrivals", "Departures"], color = "Identity")

## percentage changes

agg = x[x['Control Point'] == 'Airport'][x['Identity'] == 'Hong Kong Residents'] #iden[iden.Identity == 'Hong Kong Residents']
agg = agg.set_index('Date')
agg = agg.groupby([pd.Grouper(freq='w')]).sum()
px.bar(agg[['Departures']].pct_change(periods = 10))

## Obtain Embed